# Query a table to see what the data looks like and if it's what you want

## First, get all of your tables

In [ ]:
import sys
import shutil
from pathlib import Path

# check for existing venv, and use it if it exists. Otherwise, create fresh .venv
venv_path = Path(".venv")
if venv_path.exists():
    print("Using the existing virtual environment...")
  #  shutil.rmtree(venv_path)

else:
    print("Creating fresh virtual environment...")
    !python -m venv .venv --clear

# Use absolute paths to avoid any ambiguity
venv_abs = venv_path.absolute()
if sys.platform == "win32":
    python_exe = venv_abs / "Scripts" / "python.exe"
    pip_exe = venv_abs / "Scripts" / "pip.exe"
else:
    python_exe = venv_abs / "bin" / "python"
    pip_exe = venv_abs / "bin" / "pip"

# Install packages using explicit commands
print("\nInstalling packages...")
%pip install --upgrade pip
%pip install requests

# Test
print("\nTesting installation:")
!"{python_exe}" -c "import requests; print(f'Success! Requests from: {{requests.__file__}}')"
%pip install -r requirements.txt
# Show what's installed
print("\nInstalled packages:")
%pip list


In [ ]:
!gcloud auth application-default login
import pandas as pd
import os
from google.cloud import bigquery
from tqdm.notebook import tqdm
import time
import uuid
from datetime import datetime
import xlsxwriter

# Initialize BigQuery client
client = bigquery.Client(project="aif-usr-p-ent-ai-misc-3444")


# Set your target project and dataset
PROJECT_ID = 'aif-usr-p-ent-ai-misc-3444'  # Replace with your GCP project ID
DATASET_ID = 'column_analysis'  # The dataset name where results will be stored
LOCATION = 'US'  # Set your preferred location (e.g., 'US', 'EU')
FINAL_TABLE = 'mayo_database_summary_table'

### Optional: Get a tree of all of the tables with their columns

In [ ]:
#Interactive HTML Tree View
import pandas as pd
from IPython.display import HTML, display
import json

# Query your BigQuery table to get the results
query = f"""
SELECT
    database_name,
    table_schema,
    table_name,
    column_name,
    data_type,
    ordinal_position,
    unique_count
FROM `{PROJECT_ID}.{DATASET_ID}.{FINAL_TABLE}`
ORDER BY database_name, table_schema, table_name, ordinal_position
"""

df = client.query(query).to_dataframe()

# Create HTML tree view
def create_html_tree_view(df):
    html_content = """
    <style>
        .tree { font-family: monospace; }
        .tree ul { list-style-type: none; margin: 0; padding: 0; }
        .tree li { margin: 0; padding: 0 0 0 1.5em; }
        .tree .caret { cursor: pointer; user-select: none; }
        .tree .caret::before { content: "▶"; color: black; display: inline-block; margin-right: 6px; }
        .tree .caret-down::before { transform: rotate(90deg); }
        .tree .nested { display: none; }
        .tree .active { display: block; }
        .table-name { font-weight: bold; color: #0066cc; }
        .column-info { color: #333; }
        .unique-count { color: #666; font-style: italic; }
        .data-type { color: #008000; }
        .search-box { margin: 10px 0; padding: 8px; width: 300px; }
        .stats { margin: 10px 0; padding: 10px; background: #f0f0f0; }
    </style>

    <div class="stats">
        <h3>Database Schema Overview</h3>
        <p>Total Tables: <span id="totalTables"></span></p>
        <p>Total Columns: <span id="totalColumns"></span></p>
    </div>

    <input type="text" class="search-box" placeholder="Search tables or columns..." onkeyup="filterTree(this.value)">

    <div class="tree" id="tree">
    """

    # Group by database, schema, and table
    grouped = df.groupby(['database_name', 'table_schema', 'table_name'])

    total_tables = 0
    total_columns = len(df)

    for (db, schema, table), columns in grouped:
        total_tables += 1
        unique_columns = len(columns)

        html_content += f"""
        <ul>
            <li>
                <span class="caret table-name" data-search="{db} {schema} {table}">{db}.{schema}.{table}</span>
                <span style="color: #666;"> ({unique_columns} columns)</span>
                <ul class="nested">
        """

        for _, col in columns.iterrows():
            unique_count = col['unique_count']
            unique_display = f"unique values: {unique_count}" if unique_count != "omitted" else "counting omitted"

            html_content += f"""
                <li class="column-info" data-search="{table} {col['column_name']}">
                    <b>{col['ordinal_position']}.</b> {col['column_name']}
                    <span class="data-type">({col['data_type']})</span>
                    <span class="unique-count">[{unique_display}]</span>
                </li>
            """

        html_content += """
                </ul>
            </li>
        </ul>
        """

    html_content += f"""
    </div>

    <script>
        document.getElementById('totalTables').textContent = '{total_tables}';
        document.getElementById('totalColumns').textContent = '{total_columns}';

        var toggler = document.getElementsByClassName("caret");
        var i;

        for (i = 0; i < toggler.length; i++) {{
            toggler[i].addEventListener("click", function() {{
                this.parentElement.querySelector(".nested").classList.toggle("active");
                this.classList.toggle("caret-down");
            }});
        }}

        function filterTree(searchText) {{
            var filter = searchText.toLowerCase();
            var tree = document.getElementById("tree");
            var items = tree.getElementsByTagName("li");

            for (var i = 0; i < items.length; i++) {{
                var item = items[i];
                var searchData = item.getAttribute("data-search");

                if (searchData && searchData.toLowerCase().indexOf(filter) > -1) {{
                    item.style.display = "";
                    // Expand parent if child matches
                    var parent = item.closest(".nested");
                    if (parent) {{
                        parent.classList.add("active");
                        parent.previousElementSibling.classList.add("caret-down");
                    }}
                }} else if (item.querySelector(".caret")) {{
                    // This is a table item, check if any child matches
                    var children = item.querySelectorAll(".column-info");
                    var hasMatch = false;
                    for (var j = 0; j < children.length; j++) {{
                        var childData = children[j].getAttribute("data-search");
                        if (childData && childData.toLowerCase().indexOf(filter) > -1) {{
                            hasMatch = true;
                            break;
                        }}
                    }}
                    item.style.display = hasMatch ? "" : "none";
                }} else {{
                    item.style.display = filter === "" ? "" : "none";
                }}
            }}
        }}
    </script>
    """

    return html_content

# Display the interactive tree
html_tree = create_html_tree_view(df)
display(HTML(html_tree))


## Then, query tables of interest to see what type of data they have and what it looks like

In [ ]:


import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, FileLink
import pandas as pd

# Define database aliases and their mappings
DATABASE_ALIASES = {
    "ml-mps-adl-intudp-phi-p-d5cb": "ADL Data Assets",
    "ml-mps-adl-intfhr-phi-p-3b6e": "LPR (FHIR)"
}

# Reverse mapping for getting actual database name from alias
ALIAS_TO_DATABASE = {v: k for k, v in DATABASE_ALIASES.items()}

# Define columns to exclude
EXCLUDE_COLUMNS = [
    'ROW_ITERATION_NUMBER',
    'ROW_CURRENT_INDICATOR',
    'ROW_FROM_DTM',
    'ROW_TO_DTM',
    'ROW_SOURCE_ID',
    'ROW_LOADED_DTM'
]

# First, query your metadata table to get all available options
metadata_query = f"""
SELECT DISTINCT
    database_name,
    table_schema,
    table_name,
    COUNT(*) as column_count
FROM `{PROJECT_ID}.{DATASET_ID}.{FINAL_TABLE}`
GROUP BY database_name, table_schema, table_name
ORDER BY database_name, table_schema, table_name
"""

print("Loading available tables...")
metadata_df = client.query(metadata_query).to_dataframe()

# Create a mapping of database to its schema (assuming one schema per database)
database_schema_map = {}
for _, row in metadata_df.drop_duplicates(['database_name', 'table_schema']).iterrows():
    database_schema_map[row['database_name']] = row['table_schema']

# Get unique values for dropdowns
databases = sorted(metadata_df['database_name'].unique())
tables = sorted(metadata_df['table_name'].unique())

# Create aliases for databases in dropdown
database_display_names = []
for db in databases:
    if db in DATABASE_ALIASES:
        database_display_names.append(DATABASE_ALIASES[db])
    else:
        database_display_names.append(db)

# Create dropdown widgets
database_widget = widgets.Dropdown(
    options=list(zip(database_display_names, databases)),
    value=databases[0] if databases else '',
    description='Database:',
    style={'description_width': 'initial'}
)

table_widget = widgets.Dropdown(
    options=tables,
    value=tables[0] if tables else '',
    description='Table:',
    style={'description_width': 'initial'}
)

# Update slider range to 100-10,000
limit_widget = widgets.IntSlider(
    value=100,
    min=100,
    max=10000,
    step=100,
    description='Download Rows:',
    style={'description_width': 'initial'}
)

# Add checkbox to disable limit
no_limit_checkbox = widgets.Checkbox(
    value=False,
    description='No limit (download all rows)',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='250px')
)

# Display options
transpose_widget = widgets.Checkbox(value=False, description='Transpose')
show_info_widget = widgets.Checkbox(value=True, description='Show Info')
style_widget = widgets.Dropdown(
    options=['Default', 'Minimal', 'Detailed', 'Compact'],
    value='Default',
    description='Style:'
)

# Add checkbox for empty column exclusion
exclude_empty_widget = widgets.Checkbox(value=True, description='Exclude Empty Columns')

button = widgets.Button(description="Query Table", button_style='success')
output = widgets.Output()

# Add search box for quick table search
search_widget = widgets.Text(
    placeholder='Search tables...',
    description='Search:',
    style={'description_width': 'initial'}
)

# Add text to show current slider value and warning for unlimited
slider_value_text = widgets.HTML(value='')
warning_text = widgets.HTML(value='')

# Update UI text when slider or checkbox changes
def update_ui_text(change=None):
    if no_limit_checkbox.value:
        slider_value_text.value = '<div style="color: #666; font-size: 0.9em; margin-top: 5px;">Will download ALL rows (display shows top 20)</div>'
        warning_text.value = '<div style="color: #dc3545; font-weight: bold; font-size: 0.9em; margin-top: 5px;">⚠️ Warning: Downloading all rows may take a long time and produce large files for tables with many rows!</div>'
        limit_widget.disabled = True
    else:
        slider_value_text.value = f'<div style="color: #666; font-size: 0.9em; margin-top: 5px;">Will download {limit_widget.value} rows (display shows top 20)</div>'
        warning_text.value = ''
        limit_widget.disabled = False

limit_widget.observe(update_ui_text, names='value')
no_limit_checkbox.observe(update_ui_text, names='value')

# Initialize UI text
update_ui_text()

# Function to update dropdown options based on selections
def update_tables(change):
    """Update table dropdown based on selected database"""
    selected_db = change['new']
    selected_schema = database_schema_map[selected_db]

    filtered_tables = sorted(
        metadata_df[
            (metadata_df['database_name'] == selected_db) &
            (metadata_df['table_schema'] == selected_schema)
        ]['table_name'].unique()
    )
    table_widget.options = filtered_tables
    if filtered_tables:
        table_widget.value = filtered_tables[0]

def search_tables(change):
    """Filter all dropdowns based on search text"""
    search_text = change['new'].lower()

    if search_text:
        # Filter metadata based on search
        filtered_df = metadata_df[
            metadata_df['table_name'].str.lower().str.contains(search_text) |
            metadata_df['table_schema'].str.lower().str.contains(search_text)
        ]

        # Update dropdowns with filtered options
        filtered_dbs = sorted(filtered_df['database_name'].unique())
        filtered_tables = sorted(filtered_df['table_name'].unique())

        # Create display names for filtered databases
        filtered_db_display = []
        for db in filtered_dbs:
            if db in DATABASE_ALIASES:
                filtered_db_display.append(DATABASE_ALIASES[db])
            else:
                filtered_db_display.append(db)

        database_widget.options = list(zip(filtered_db_display, filtered_dbs))
        table_widget.options = filtered_tables

        # Set first values if available
        if filtered_dbs:
            database_widget.value = filtered_dbs[0]
        if filtered_tables:
            table_widget.value = filtered_tables[0]
    else:
        # Reset to all options
        database_widget.options = list(zip(database_display_names, databases))
        table_widget.options = tables

# Attach event handlers
database_widget.observe(update_tables, names='value')
search_widget.observe(search_tables, names='value')

# Add quick select buttons for common tables
quick_select_buttons = []
common_tables = metadata_df.nlargest(5, 'column_count')  # Top 5 tables by column count

for _, row in common_tables.iterrows():
    # Get display name for database
    db_display = DATABASE_ALIASES.get(row['database_name'], row['database_name'])

    btn = widgets.Button(
        description=f"{row['table_name'][:20]}... ({row['column_count']} cols)",
        layout=widgets.Layout(width='auto'),
        button_style='info'
    )

    # Create a closure to capture the row values
    def make_handler(db, table):
        def handler(b):
            database_widget.value = db
            table_widget.value = table
        return handler

    btn.on_click(make_handler(row['database_name'], row['table_name']))
    quick_select_buttons.append(btn)

def get_filtered_columns(database, schema, table, exclude_empty=True):
    """Get list of columns for a table, excluding system columns and optionally empty columns"""
    # Query the metadata table to get column info including unique counts
    column_query = f"""
    SELECT
        column_name,
        unique_count,
        data_type
    FROM `{PROJECT_ID}.{DATASET_ID}.{FINAL_TABLE}`
    WHERE
        database_name = '{database}'
        AND table_schema = '{schema}'
        AND table_name = '{table}'
    ORDER BY ordinal_position
    """

    try:
        columns_df = client.query(column_query).to_dataframe()

        # Start with all columns
        filtered_columns = columns_df['column_name'].tolist()

        # Filter out system columns
        filtered_columns = [col for col in filtered_columns if col not in EXCLUDE_COLUMNS]

        # Filter out empty columns if requested
        if exclude_empty:
            empty_columns = []
            for _, row in columns_df.iterrows():
                column_name = row['column_name']
                unique_count = str(row['unique_count'])

                # Skip if already excluded as system column
                if column_name not in filtered_columns:
                    continue

                # Check if column should be considered empty
                if (unique_count == "0" or
                    (unique_count == "1" and column_name not in ['ROW_CURRENT_INDICATOR'])):  # Single value might be NULL/empty
                    empty_columns.append(column_name)

            # Remove empty columns
            filtered_columns = [col for col in filtered_columns if col not in empty_columns]

            if empty_columns:
                print(f"Excluded empty columns: {', '.join(empty_columns)}")

        return filtered_columns, columns_df
    except Exception as e:
        print(f"Error getting column metadata: {e}")
        # If we can't get column info, return None to use SELECT *
        return None, None

# Optional: Add a table info section
info_output = widgets.Output()

def show_table_info(change):
    """Show information about the selected table"""
    with info_output:
        clear_output()

        selected_db = database_widget.value
        selected_schema = database_schema_map[selected_db]

        # Get full metadata for the selected table
        table_metadata_query = f"""
        SELECT
            column_name,
            data_type,
            unique_count
        FROM `{PROJECT_ID}.{DATASET_ID}.{FINAL_TABLE}`
        WHERE
            database_name = '{selected_db}'
            AND table_schema = '{selected_schema}'
            AND table_name = '{table_widget.value}'
        ORDER BY ordinal_position
        """

        try:
            table_metadata = client.query(table_metadata_query).to_dataframe()

            print(f"Table: {table_widget.value}")
            print(f"Total Columns: {len(table_metadata)}")

            # Count columns by type
            empty_count = len(table_metadata[
                (table_metadata['unique_count'].astype(str) == "0") |
                (table_metadata['unique_count'].astype(str) == "1")
            ])

            system_count = len([col for col in table_metadata['column_name'] if col in EXCLUDE_COLUMNS])

            print(f"System Columns: {system_count}")
            print(f"Empty/Single-Value Columns: {empty_count}")
            print(f"Data Columns: {len(table_metadata) - system_count - empty_count}")

            if exclude_empty_widget.value:
                print("\nNote: Empty columns will be excluded from query")

        except Exception as e:
            print(f"Error loading table metadata: {e}")

# Add custom CSS for better table display with improved button states
display(HTML("""
<style>
    .table-container {
        overflow-x: auto;
        overflow-y: auto;
        max-height: 600px;
        border: 1px solid #ddd;
        padding: 10px;
        margin: 10px 0;
    }

    .dataframe {
        font-size: 12px;
        border-collapse: collapse;
        width: auto;
    }

    .dataframe th {
        position: sticky;
        top: 0;
        background-color: #f0f0f0;
        padding: 8px;
        text-align: center;
        border: 1px solid #ddd;
        font-weight: bold;
        z-index: 10;
    }

    .dataframe td {
        padding: 6px;
        border: 1px solid #ddd;
        max-width: 300px;
        overflow: hidden;
        text-overflow: ellipsis;
        white-space: nowrap;
    }

    .dataframe tr:nth-child(even) {
        background-color: #f9f9f9;
    }

    .dataframe tr:hover {
        background-color: #f5f5f5;
    }

    .column-toggle {
        margin: 10px 0;
        padding: 10px;
        background-color: #f8f8f8;
        border: 1px solid #ddd;
        border-radius: 5px;
    }

    .column-toggle button {
        margin: 2px;
        padding: 5px 10px;
        font-size: 11px;
        cursor: pointer;
        border: 1px solid #ccc;
        background-color: #e0e0e0;
        border-radius: 3px;
        transition: all 0.2s ease;
    }

    /* Active (shown) columns */
    .column-toggle button.active {
        background-color: #4CAF50;
        color: white;
        border-color: #45a049;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }

    /* Inactive (hidden) columns */
    .column-toggle button.inactive {
        background-color: #f5f5f5;
        color: #999;
        border-color: #ddd;
        text-decoration: line-through;
        opacity: 0.7;
    }

    .column-toggle button:hover {
        transform: translateY(-1px);
        box-shadow: 0 2px 5px rgba(0,0,0,0.2);
    }

    .controls-panel {
        margin: 10px 0;
        padding: 10px;
        background-color: #f0f0f0;
        border-radius: 5px;
    }

    .control-button {
        margin-right: 10px;
        padding: 8px 16px;
        background-color: #007bff;
        color: white;
        border: none;
        border-radius: 4px;
        cursor: pointer;
    }

    .control-button:hover {
        background-color: #0056b3;
    }

    .control-button.danger {
        background-color: #dc3545;
    }

    .control-button.danger:hover {
        background-color: #c82333;
    }

    .control-button.success {
        background-color: #28a745;
    }

    .control-button.success:hover {
        background-color: #218838;
    }
</style>
"""))

# Global variable to store download data
download_data_df = None

# Enhanced query function with better display
def query_with_options_enhanced(b):
    global download_data_df

    with output:
        clear_output()

        # Get the selected database and its corresponding schema
        selected_db = database_widget.value
        selected_schema = database_schema_map[selected_db]

        # Build full table name
        full_table_name = f"{selected_db}.{selected_schema}.{table_widget.value}"

        # Get filtered columns
        filtered_columns, columns_metadata = get_filtered_columns(
            selected_db,
            selected_schema,
            table_widget.value,
            exclude_empty=exclude_empty_widget.value
        )

        # Build query with filtered columns
        if filtered_columns:
            column_list = ', '.join([f"`{col}`" for col in filtered_columns])
        else:
            column_list = '*'

        # Display query (always limit to 20)
        display_query = f"""
        SELECT {column_list}
        FROM `{full_table_name}`
        LIMIT 20
        """

        # Download query (with or without limit)
        if no_limit_checkbox.value:
            download_query = f"""
            SELECT {column_list}
            FROM `{full_table_name}`
            """
            download_limit_text = "ALL"
        else:
            download_query = f"""
            SELECT {column_list}
            FROM `{full_table_name}`
            LIMIT {limit_widget.value}
            """
            download_limit_text = f"{limit_widget.value}"

        try:
            # Get display name for database
            db_display = DATABASE_ALIASES.get(selected_db, selected_db)
            print(f"Querying: {db_display}.{selected_schema}.{table_widget.value}")

            # Execute display query
            df = client.query(display_query).to_dataframe()

            # Execute download query
            print(f"Preparing download data ({download_limit_text} rows)...")
            download_data_df = client.query(download_query).to_dataframe()

            print(f"Display dataframe shape: {df.shape[0]} rows × {df.shape[1]} columns")
            print(f"Download dataframe shape: {download_data_df.shape[0]} rows × {download_data_df.shape[1]} columns")

            # Additional filtering in case SELECT * was used
            if filtered_columns is None:
                columns_to_drop = [col for col in df.columns if col in EXCLUDE_COLUMNS]
                if columns_to_drop:
                    df = df.drop(columns=columns_to_drop)
                    download_data_df = download_data_df.drop(columns=columns_to_drop)
                    print(f"Excluded system columns: {', '.join(columns_to_drop)}")

            if show_info_widget.value:
                print(f"Display Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
                print(f"Download Memory: {download_data_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
                print()

            # Display the table (only showing top 20)
            if style_widget.value == 'Minimal':
                display(df)
            else:
                # Create the table with enhanced styling
                table_html = df.to_html(
                    classes='dataframe current-table',
                    index=False,
                    escape=False,
                    max_rows=None,
                    max_cols=None
                )

                # Wrap in a scrollable container
                full_html = f"""
                <div class="table-container">
                    <p style="font-weight: bold; margin-bottom: 10px;">
                        Displaying top 20 rows (will download {download_limit_text} rows)
                    </p>
                    {table_html}
                </div>
                """

                display(HTML(full_html))

            # Create column toggle controls with JavaScript
            if style_widget.value != 'Minimal':
                controls_html = """
                <div class="controls-panel">
                    <h4>Column Controls</h4>
                    <button class="control-button success" onclick="showAllColumns()">Show All</button>
                    <button class="control-button danger" onclick="hideAllColumns()">Hide All</button>
                    <button class="control-button" onclick="resetColumnWidths()">Reset Width</button>
                </div>

                <div class="column-toggle">
                """

                for idx, col in enumerate(df.columns):
                    controls_html += f'<button id="col-{idx}" class="active" onclick="toggleColumn({idx})">{col}</button>'

                controls_html += """
                </div>

                <script>
                    function toggleColumn(colIndex) {
                        var table = document.querySelector('.current-table');
                        if (!table) return;

                        var button = document.getElementById('col-' + colIndex);
                        var rows = table.querySelectorAll('tr');
                        var isHidden = button.classList.contains('inactive');

                        for (var i = 0; i < rows.length; i++) {
                            var cells = rows[i].querySelectorAll('td, th');
                            if (cells[colIndex]) {
                                if (isHidden) {
                                    cells[colIndex].style.display = '';
                                    button.classList.remove('inactive');
                                    button.classList.add('active');
                                } else {
                                    cells[colIndex].style.display = 'none';
                                    button.classList.remove('active');
                                    button.classList.add('inactive');
                                }
                            }
                        }
                    }

                    function showAllColumns() {
                        var table = document.querySelector('.current-table');
                        if (!table) return;

                        var rows = table.querySelectorAll('tr');
                        var buttons = document.querySelectorAll('.column-toggle button');

                        for (var i = 0; i < rows.length; i++) {
                            var cells = rows[i].querySelectorAll('td, th');
                            for (var j = 0; j < cells.length; j++) {
                                cells[j].style.display = '';
                            }
                        }

                        buttons.forEach(function(button) {
                            button.classList.remove('inactive');
                            button.classList.add('active');
                        });
                    }

                    function hideAllColumns() {
                        var table = document.querySelector('.current-table');
                        if (!table) return;

                        var rows = table.querySelectorAll('tr');
                        var buttons = document.querySelectorAll('.column-toggle button');

                        for (var i = 0; i < rows.length; i++) {
                            var cells = rows[i].querySelectorAll('td, th');
                            for (var j = 0; j < cells.length; j++) {
                                cells[j].style.display = 'none';
                            }
                        }

                        buttons.forEach(function(button) {
                            button.classList.remove('active');
                            button.classList.add('inactive');
                        });
                    }

                    function resetColumnWidths() {
                        var cells = document.querySelectorAll('.current-table td, .current-table th');
                        cells.forEach(function(cell) {
                            cell.style.maxWidth = '300px';
                            cell.style.width = '';
                        });
                    }
                </script>
                """

                display(HTML(controls_html))

            # Add export functionality
            export_controls = widgets.HBox([
                widgets.Button(description="Download CSV", button_style='primary'),
                widgets.Button(description="Download Excel", button_style='primary')
            ])

            export_output = widgets.Output()

            def download_csv(b):
                with export_output:
                    clear_output()
                    try:
                        # Create a filename based on table name and limit
                        limit_suffix = "all" if no_limit_checkbox.value else f"{limit_widget.value}"
                        filename = f"{table_widget.value}_{limit_suffix}_rows.csv"

                        # Save to current working directory
                        download_data_df.to_csv(filename, index=False)

                        # Display download link using FileLink
                        display(FileLink(filename, result_html_prefix=f"✅ CSV file saved: "))
                        print(f"File contains {download_data_df.shape[0]} rows × {download_data_df.shape[1]} columns")

                    except Exception as e:
                        print(f"❌ Error saving CSV: {e}")
            # Download the excel doc
            def download_excel(b):
                with export_output:
                    clear_output()
                    try:
                        # Create a filename based on table name and limit
                        limit_suffix = "all" if no_limit_checkbox.value else f"{limit_widget.value}"
                        filename = f"{table_widget.value}_{limit_suffix}_rows.xlsx"

                        # Create a Pandas Excel writer using XlsxWriter as the engine
                        with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
                            # Convert the dataframe to an XlsxWriter Excel object
                            download_data_df.to_excel(writer, sheet_name='Data', index=False)
                            
                            # Get the xlsxwriter workbook and worksheet objects
                            workbook = writer.book
                            worksheet = writer.sheets['Data']
                            
                            # Set column widths to ensure all data is visible
                            for i, col in enumerate(download_data_df.columns):
                                # Set column width based on content
                                max_len = max(
                                    download_data_df[col].astype(str).str.len().max(),
                                    len(str(col))
                                ) + 2
                                worksheet.set_column(i, i, min(max_len, 50))  # Cap width at 50

                        # Display download link using FileLink
                        display(FileLink(filename, result_html_prefix=f"✅ Excel file saved: "))
                        print(f"File contains {download_data_df.shape[0]} rows × {download_data_df.shape[1]} columns")
                        print(f"All {len(download_data_df.columns)} columns should be visible in the Excel file")

                    except Exception as e:
                        print(f"❌ Error saving Excel: {e}")
                        print(f"Debug info: DataFrame has shape {download_data_df.shape}")
                        # Print first few column names to verify data is correct
                        print(f"First 5 columns: {list(download_data_df.columns[:5])}")

            export_controls.children[0].on_click(download_csv)
            export_controls.children[1].on_click(download_excel)

            display(HTML('<div class="controls-panel"><h4>Export Options</h4></div>'))
            display(export_controls)
            display(export_output)

        except Exception as e:
            print(f"Error: {e}")

# Replace the original button click handler
button.on_click(query_with_options_enhanced)

# Initialize the dropdowns with correct filtering
update_tables({'new': database_widget.value})

# Create the limit controls
limit_controls = widgets.HBox([
    limit_widget,
    no_limit_checkbox
], layout=widgets.Layout(margin='10px 0'))

# Display interface
display(widgets.VBox([
    widgets.HTML("""
    <div style="background-color: #f8f9fa; padding: 20px; border-radius: 10px; margin-bottom: 20px;">
        <h2 style="color: #333; margin-top: 0;">🔍 BigQuery Table Explorer</h2>
        <p style="color: #666;">Explore your BigQuery tables with interactive controls</p>
    </div>
    """),
    widgets.HTML('<div style="padding: 10px;">'),
    search_widget,
    widgets.HTML("<b>Quick Select:</b>"),
    widgets.HBox(quick_select_buttons),
    widgets.HTML("<hr>"),
    widgets.HBox([database_widget, table_widget]),
    limit_controls,
    slider_value_text,
    warning_text,
    widgets.HBox([transpose_widget, show_info_widget, style_widget, exclude_empty_widget]),
    button,
    widgets.HTML('</div>'),
    output
]))

table_widget.observe(show_table_info, names='value')
display(info_output)